<a href="https://colab.research.google.com/github/patrickdevv/FIAP_Fase02_Desafio/blob/main/Algoritimo_Gen%C3%A9tico_em_forma%C3%A7%C3%A3o_de_carteira_de_a%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Análise das 10 acões mais negociadas na bolsa em 2024 a fim de gerar uma carteira de investimento

## Limpeza dos dados



1. Ler o arquivo .txt contendo as cotações.
2. Filtrar as cotações das ações especificadas (VALE3, PETR4, ITUB4, PETR3, BBDC4)
3. Salvar o resultado filtrado em um novo arquivo.

In [11]:
import csv
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import random

In [14]:
def merge_and_filter_stock_files(file_2023, file_2024, output_file, stock_codes, start_date, end_date):
    # Define os cabeçalhos reduzidos
    headers = ['Código de Negociação', 'Data de Pregão', 'Preço Último Negociado', 'Volume Total Negociado']

    # Abre os arquivos de entrada e o arquivo de saída CSV
    with open(file_2023, 'r') as file1, open(file_2024, 'r') as file2, open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)  # Escreve os cabeçalhos no CSV

        # Mescla e filtra o conteúdo dos arquivos
        for line in file1:
            process_line(line, stock_codes, start_date, end_date, writer)
        for line in file2:
            process_line(line, stock_codes, start_date, end_date, writer)

def process_line(line, stock_codes, start_date, end_date, writer):
    # Verifica se é o registro de tipo '01' (dados de cotações diárias)
    if line.startswith('01'):
        # Extrai os campos conforme o layout do arquivo
        stock_code = line[12:24].strip()
        stock_date = line[2:10]

        # Verifica se o código de negociação está na lista de ações de interesse e se a data está no intervalo
        if stock_code in stock_codes and start_date <= stock_date[:6] <= end_date:
            # Extrai apenas as colunas necessárias
            row = [
                line[12:24].strip(),   # Código de Negociação
                line[2:10],            # Data de Pregão
                line[108:121].strip(), # Preço Último Negociado
                line[170:188].strip()  # Volume Total Negociado
            ]
            writer.writerow(row)  # Escreve a linha filtrada no CSV

# Definição dos códigos de negociação para as ações de interesse
stock_codes = ['VALE3', 'PETR4', 'ITUB4', 'PETR3', 'BBDC4','BBAS3','ELET3', 'B3SA3', 'WEGE3', 'SBSP3']

# Definição do intervalo de datas no formato YYYYMM
start_date = '202309'
end_date = '202408'

# Caminhos para os arquivos de entrada e saída
file_2023 = '/content/drive/MyDrive/Pós Graduação /Pós Tech - IA para DEVs/Desafio 2/COTAHIST_A2023.TXT'  # Substitua pelo caminho do arquivo de 2023
file_2024 = '/content/drive/MyDrive/Pós Graduação /Pós Tech - IA para DEVs/Desafio 2/COTAHIST_A2024.TXT'  # Substitua pelo caminho do arquivo de 2024
output_file = 'filtered_stocks_genetic_algo.csv'

# Processa os arquivos, unindo-os e filtrando por ações e data, e salva em CSV
merge_and_filter_stock_files(file_2023, file_2024, output_file, stock_codes, start_date, end_date)

print(f'Cotações filtradas e salvas em {output_file}.')

Cotações filtradas e salvas em filtered_stocks_genetic_algo.csv.


In [16]:
acoes = pd.read_csv('filtered_stocks_genetic_algo.csv')
acoes.head()

,Código de Negociação,Data de Pregão,Preço Último Negociado,Volume Total Negociado
0,ELET3,20230901,3539,21828802000
1,PETR3,20230901,3569,107579666400
2,PETR4,20230901,3263,162115985200
3,WEGE3,20230901,3595,37269167100
4,VALE3,20230901,6889,389654873400


In [21]:
import numpy as np
import pandas as pd
import random

# Função para calcular o retorno esperado e o risco (volatilidade)
def portfolio_performance(weights, returns):
    portfolio_return = np.sum(returns.mean() * weights) * 252  # Retorno anualizado
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))  # Risco anualizado
    return portfolio_return, portfolio_risk

# Função de aptidão para o algoritmo genético (balanceia retorno e risco)
def fitness_function(weights, returns, risk_free_rate=0.01):
    ret, vol = portfolio_performance(weights, returns)
    sharpe_ratio = (ret - risk_free_rate) / vol
    return sharpe_ratio  # Queremos maximizar o Sharpe Ratio

# Função de restrição: somar os pesos igual a 1 e garantir que nenhum peso exceda 20%
def check_constraints(weights):
    return np.sum(weights) - 1.0  # A soma dos pesos deve ser 1

# Ajusta os pesos para garantir que nenhum ativo tenha mais de 20%
def adjust_weights(weights):
    max_weight = 0.20
    while np.any(weights > max_weight):
        # Encontrar onde os pesos excedem 20%
        over_limit = weights > max_weight
        excess = np.sum(weights[over_limit] - max_weight)
        weights[over_limit] = max_weight
        # Redistribuir o excesso proporcionalmente entre os pesos restantes
        weights[~over_limit] += excess / np.sum(~over_limit)
    return weights

# Função que roda o algoritmo genético com restrições
def genetic_algorithm_optimization(returns, population_size=100, generations=500, mutation_rate=0.01):
    population = np.random.dirichlet(np.ones(len(returns.columns)), size=population_size)  # População inicial

    # Evolução
    for generation in range(generations):
        fitness_scores = [fitness_function(indiv, returns) for indiv in population]
        selected_population = [population[i] for i in np.argsort(fitness_scores)[-population_size//2:]]

        offspring = []
        for _ in range(population_size - len(selected_population)):
            parents = random.sample(selected_population, 2)
            crossover_point = random.randint(1, len(returns.columns) - 1)
            child = np.concatenate((parents[0][:crossover_point], parents[1][crossover_point:]))

            # Ajustar pesos para garantir que nenhum ativo tenha mais de 20%
            child = adjust_weights(child)

            offspring.append(child)

        for indiv in offspring:
            if random.random() < mutation_rate:
                mutation_index = random.randint(0, len(returns.columns) - 1)
                indiv[mutation_index] = np.random.uniform(0, 0.2)

            # Garantir que o portfólio respeite a regra de 20%
            indiv = adjust_weights(indiv)

        population = selected_population + offspring

    fitness_scores = [fitness_function(indiv, returns) for indiv in population]
    best_individual = population[np.argmax(fitness_scores)]

    return best_individual

# Carregar dados de retorno dos ativos
data = pd.read_csv('filtered_stocks_genetic_algo.csv')

# Calculando os retornos diários com base nos preços de fechamento
data['Returns'] = data.groupby('Código de Negociação')['Preço Último Negociado'].pct_change()

# Cria uma matriz de retornos, onde as colunas são os ativos e as linhas as datas
returns = data.pivot(index='Data de Pregão', columns='Código de Negociação', values='Returns')

# Limitações de peso para cada ativo (nenhum com mais de 20%)
bounds = [(0, 0.2)] * len(returns.columns)

# Rodar o algoritmo genético com a restrição corrigida
best_portfolio = genetic_algorithm_optimization(returns)

# Exibir o portfólio ideal
portfolio_return, portfolio_risk = portfolio_performance(best_portfolio, returns)

# Associar os pesos aos ativos (códigos de negociação)
portfolio_allocation = pd.Series(best_portfolio, index=returns.columns)

# Exibir a alocação ideal de ativos
print("Alocação de Ativos Ideal no Portfólio:")
print(portfolio_allocation)

# Exibir retorno e risco do portfólio
print(f"\nRetorno esperado anualizado: {portfolio_return}")
print(f"Risco anualizado: {portfolio_risk}")


Alocação de Ativos Ideal no Portfólio:
Código de Negociação
B3SA3    0.002447
BBAS3    0.000399
BBDC4    0.005418
ELET3    0.002242
ITUB4    0.198238
PETR3    0.014512
PETR4    0.084054
SBSP3    0.200000
VALE3    0.004742
WEGE3    0.187121
dtype: float64

Retorno esperado anualizado: 0.2710943188919792
Risco anualizado: 0.09470866008235684


## Implementação dos Testes

1. Testes com o Algoritmo Genético
Executaremos o algoritmo genético com os dados de retorno dos ativos e analisaremos os resultados com base em:

Retorno esperado anualizado
Risco anualizado (volatilidade)
Sharpe Ratio (índice que avalia o retorno ajustado ao risco)
2. Comparação com Método Convencional
Para comparação, usaremos um método tradicional amplamente conhecido em otimização de portfólio, como a Otimização Quadrática de Markowitz, que encontra a fronteira eficiente de portfólios. Esse método utiliza a covariância entre os ativos e os retornos esperados para encontrar a melhor combinação entre retorno e risco.

3. Métricas de Comparação
Vamos comparar o portfólio otimizado pelo algoritmo genético com o portfólio otimizado pelo método quadrático em termos de:

Retorno esperado anualizado
Risco anualizado
Sharpe Ratio


Abaixo está o código para realizar esses testes e comparações.

Passo 1: Otimização com o Algoritmo Genético
Vamos reutilizar o algoritmo genético que desenvolvemos anteriormente.

Passo 2: Otimização Quadrática Convencional (Markowitz)
Adicionaremos a otimização quadrática para comparar com a solução genética.

In [22]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import random

# Função para calcular o retorno esperado e o risco (volatilidade)
def portfolio_performance(weights, returns):
    portfolio_return = np.sum(returns.mean() * weights) * 252  # Retorno anualizado
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))  # Risco anualizado
    return portfolio_return, portfolio_risk

# Função de aptidão para o algoritmo genético (balanceia retorno e risco)
def fitness_function(weights, returns, risk_free_rate=0.01):
    ret, vol = portfolio_performance(weights, returns)
    sharpe_ratio = (ret - risk_free_rate) / vol
    return sharpe_ratio  # Queremos maximizar o Sharpe Ratio

# Função de restrição: somar os pesos igual a 1
def check_constraints(weights):
    return np.sum(weights) - 1.0  # A soma dos pesos deve ser 1

# Função que roda o algoritmo genético
def genetic_algorithm_optimization(returns, population_size=100, generations=500, mutation_rate=0.01):
    population = np.random.dirichlet(np.ones(len(returns.columns)), size=population_size)  # População inicial

    # Evolução
    for generation in range(generations):
        fitness_scores = [fitness_function(indiv, returns) for indiv in population]
        selected_population = [population[i] for i in np.argsort(fitness_scores)[-population_size//2:]]

        offspring = []
        for _ in range(population_size - len(selected_population)):
            parents = random.sample(selected_population, 2)
            crossover_point = random.randint(1, len(returns.columns) - 1)
            child = np.concatenate((parents[0][:crossover_point], parents[1][crossover_point:]))
            offspring.append(child)

        for indiv in offspring:
            if random.random() < mutation_rate:
                mutation_index = random.randint(0, len(returns.columns) - 1)
                indiv[mutation_index] = np.random.uniform(0, 0.2)

        population = selected_population + offspring

    fitness_scores = [fitness_function(indiv, returns) for indiv in population]
    best_individual = population[np.argmax(fitness_scores)]

    return best_individual

# Função para otimização de Markowitz (método convencional)
def markowitz_optimization(returns):
    num_assets = len(returns.columns)

    def objective_function(weights):
        return -fitness_function(weights, returns)

    initial_weights = np.ones(num_assets) / num_assets  # Pesos iniciais iguais
    bounds = [(0, 0.2)] * num_assets  # Limite de 20% por ativo
    constraints = {'type': 'eq', 'fun': check_constraints}  # Restrição: soma dos pesos = 1

    result = minimize(objective_function, initial_weights, bounds=bounds, constraints=constraints)

    return result.x  # Retorna os pesos otimizados

# Carregar dados de retorno dos ativos
data = pd.read_csv('filtered_stocks_genetic_algo.csv')

# Calculando os retornos diários com base nos preços de fechamento
data['Returns'] = data.groupby('Código de Negociação')['Preço Último Negociado'].pct_change()

# Cria uma matriz de retornos, onde as colunas são os ativos e as linhas as datas
returns = data.pivot(index='Data de Pregão', columns='Código de Negociação', values='Returns')

# Limitações de peso para cada ativo (nenhum com mais de 20%)
bounds = [(0, 0.2)] * len(returns.columns)

# Otimização com algoritmo genético
best_portfolio_genetic = genetic_algorithm_optimization(returns)

# Otimização com o método de Markowitz
best_portfolio_markowitz = markowitz_optimization(returns)

# Exibir os portfólios
print("\nPortfólio Otimizado com Algoritmo Genético:")
portfolio_allocation_genetic = pd.Series(best_portfolio_genetic, index=returns.columns)
print(portfolio_allocation_genetic)

print("\nPortfólio Otimizado com Método de Markowitz:")
portfolio_allocation_markowitz = pd.Series(best_portfolio_markowitz, index=returns.columns)
print(portfolio_allocation_markowitz)

# Performance do portfólio genético
portfolio_return_genetic, portfolio_risk_genetic = portfolio_performance(best_portfolio_genetic, returns)
print(f"\nDesempenho do Portfólio Genético: Retorno anualizado = {portfolio_return_genetic}, Risco anualizado = {portfolio_risk_genetic}")

# Performance do portfólio Markowitz
portfolio_return_markowitz, portfolio_risk_markowitz = portfolio_performance(best_portfolio_markowitz, returns)
print(f"\nDesempenho do Portfólio Markowitz: Retorno anualizado = {portfolio_return_markowitz}, Risco anualizado = {portfolio_risk_markowitz}")

# Comparação de Sharpe Ratio
sharpe_genetic = (portfolio_return_genetic - 0.01) / portfolio_risk_genetic
sharpe_markowitz = (portfolio_return_markowitz - 0.01) / portfolio_risk_markowitz

print(f"\nSharpe Ratio - Algoritmo Genético: {sharpe_genetic}")
print(f"Sharpe Ratio - Método Markowitz: {sharpe_markowitz}")



Portfólio Otimizado com Algoritmo Genético:
Código de Negociação
B3SA3    0.000313
BBAS3    0.006867
BBDC4    0.009256
ELET3    0.003483
ITUB4    0.194141
PETR3    0.063287
PETR4    0.041125
SBSP3    0.193064
VALE3    0.017275
WEGE3    0.197340
dtype: float64

Portfólio Otimizado com Método de Markowitz:
Código de Negociação
B3SA3    0.000000
BBAS3    0.000000
BBDC4    0.043078
ELET3    0.104345
ITUB4    0.200000
PETR3    0.052577
PETR4    0.200000
SBSP3    0.200000
VALE3    0.000000
WEGE3    0.200000
dtype: float64

Desempenho do Portfólio Genético: Retorno anualizado = 0.26931555689203546, Risco anualizado = 0.0973658611473507

Desempenho do Portfólio Markowitz: Retorno anualizado = 0.33776592575733133, Risco anualizado = 0.13627473371414087

Sharpe Ratio - Algoritmo Genético: 2.663310875457618
Sharpe Ratio - Método Markowitz: 2.405184855799281


## Comparação Final:

### Retorno Anualizado:
O portfólio de Markowitz apresenta um retorno esperado mais alto (33,78% contra 26,93% do algoritmo genético), mas isso vem com um custo de maior risco.
### Risco Anualizado (Volatilidade):
O portfólio genético tem um risco significativamente menor (9,74% contra 13,63% de Markowitz), o que indica que é uma opção mais estável e menos sujeita a grandes oscilações.
###Sharpe Ratio:
O Sharpe Ratio do portfólio genético é mais alto (2.66 contra 2.40), o que significa que o portfólio gerado pelo algoritmo genético oferece uma melhor relação risco-retorno. Em outras palavras, ele está gerando mais retorno por unidade de risco em comparação com o método de Markowitz.

## Conclusão:
### Portfólio Genético:
Tem um Sharpe Ratio superior, indicando que ele é mais eficiente na geração de retorno ajustado ao risco. Ele é uma escolha sólida para um investidor que prefere minimizar a volatilidade e ainda obter um bom retorno.

### Portfólio Markowitz:
Oferece um retorno anualizado superior, mas vem com um risco mais alto. É mais indicado para investidores dispostos a aceitar uma maior volatilidade em troca de um retorno potencialmente maior.

